# Import

In [1]:
import numpy as np
import healpy as hp
import pandas as pd
import networkx as nx
import geopandas as gpd

from itertools import combinations

from pyproj import Geod
from shapely import union_all, intersects
from shapely.wkt import loads
from shapely.geometry import LineString
from healpix_geo.nested import healpix_to_lonlat
from healpix_geo.nested import lonlat_to_healpix

import plotly.express as px
import plotly.graph_objects as go

import matplotlib.pyplot as plt

In [2]:
def get_points(NSIDE):
    NPIX = hp.nside2npix(NSIDE)
    points = np.array(
        hp.pix2vec(NSIDE, range(NPIX), nest=True)
    ).T

    df = pd.DataFrame(points, columns=('x', 'y', 'z'))

    return df

def get_patches(NSIDE):
    NPIX = hp.nside2npix(NSIDE)

    pixels = np.arange(NPIX)
    patches = np.array(
        [hp.boundaries(NSIDE, i, nest=True).T for i in pixels]
    ).reshape(-1, 3)

    df = pd.DataFrame(patches, columns=('x', 'y', 'z'))

    return df

In [ ]:
def great_circle_path(a=None, b=None, n_points=100, iterable=False):
    geod = Geod(ellps='WGS84')

    if iterable:
        (a, b) = a

    points = geod.npts(
        a.x, a.y,
        b.x, b.y,
        n_points,
    )
    line = LineString(points)

    return line

def great_circle_distance(a=None, b=None, iterable=False):
    geod = Geod(ellps='WGS84')

    if iterable:
        (a, b) = a

    distance = geod.inv(
        a.x, a.y,
        b.x, b.y,
    )[2]

    return distance

# Data

## Borders

In [3]:
kaz = gpd.read_file('data/geoBoundaries-KAZ-ADM0_simplified.geojson')
ukr = gpd.read_file('data/geoBoundaries-UKR-ADM0_simplified.geojson')
rus = gpd.read_file('data/geoBoundaries-RUS-ADM0_simplified.geojson')
blr = gpd.read_file('data/geoBoundaries-BLR-ADM0_simplified.geojson')
borders = pd.concat((kaz, ukr, rus, blr)).reset_index(drop=True)
borders = borders.to_crs(epsg=4326)
borders

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,Kazakhstan,KAZ,12445969B25401373275960,KAZ,ADM0,"MULTIPOLYGON (((50.3741 44.78639, 50.37386 44...."
1,Ukraine,UKR,60985579B58127766181095,UKR,ADM0,"POLYGON ((33.19618 52.37361, 33.19572 52.37895..."
2,Russian Federation,RUS,89588193B69022144921215,RUS,ADM0,"MULTIPOLYGON (((130.82214 42.4629, 130.8202 42..."
3,Belarus,BLR,10690812B6439167788480,BLR,ADM0,"POLYGON ((31.78388 52.10805, 31.78611 52.12416..."


## Airports

In [4]:
airports = pd.read_csv('data/airports.csv')
airports = gpd.GeoDataFrame(
    airports.drop(['GeoPointLat', 'GeoPointLong'], axis=1),
    geometry=gpd.points_from_xy(
        *airports[['GeoPointLong', 'GeoPointLat']].values.T
    ), crs='EPSG:4326'
)
airports[airports.ICAO.isin(['KJFK', 'UAAA'])]

,AirportName,IATA,ICAO,TimeZone,City_Name,City_IATA,UTC_Offset_Hours,UTC_Offset_Seconds,Country_CodeA2,Country_CodeA3,Country_Name,geometry
2169,John F Kennedy Intl,JFK,KJFK,America/New_York,"New York, New York",NYC,-4.0,-14400.0,US,USA,United States of America,POINT (-73.77892 40.63975)
5300,Almaty,ALA,UAAA,Asia/Qyzylorda,Almaty,ALA,5.0,18000.0,KZ,KAZ,Kazakhstan,POINT (77.04051 43.35207)


# HEALPix

In [5]:
NSIDE = 16
healpix = get_points(NSIDE)

NPIX = hp.nside2npix(NSIDE)
ipix = np.arange(NPIX)
depth = np.log2(NSIDE).astype(int)

print(NPIX, depth)

3072 4


In [7]:
df = pd.read_csv('point_pairs.csv', index_col=False)

df = df.rename({'0': 'point_a', '1': 'point_b'}, axis=1)
df = df.drop(['Unnamed: 0'], axis=1)

df.loc[:, 'point_a'] = loads(df.loc[:, 'point_a'])
df.loc[:, 'point_b'] = loads(df.loc[:, 'point_b'])

df['healpix_id_a'] = lonlat_to_healpix(
    gpd.GeoSeries(df.loc[:, 'point_a']).geometry.x,
    gpd.GeoSeries(df.loc[:, 'point_a']).geometry.y,
    depth,
    ellipsoid='WGS84',
)
df['healpix_id_b'] = lonlat_to_healpix(
    gpd.GeoSeries(df.loc[:, 'point_b']).geometry.x,
    gpd.GeoSeries(df.loc[:, 'point_b']).geometry.y,
    depth,
    ellipsoid='WGS84',
)

df

,point_a,point_b,path,intersect_avoid,healpix_id_a,healpix_id_b
0,POINT (45 2.3987250992841447),POINT (47.8125 4.801554594177967),LINESTRING (45.055020515638404 2.4459152101302...,False,0,1
1,POINT (45 2.3987250992841447),POINT (42.1875 4.801554594177967),LINESTRING (44.944979484361596 2.4459152101302...,False,0,2
2,POINT (45 2.3987250992841447),POINT (45 7.212658108949127),"LINESTRING (45 2.4931214185652424, 45 2.587517...",False,0,3
3,POINT (45 2.3987250992841447),POINT (50.625 7.212658108949127),LINESTRING (45.109656950531374 2.4934954503132...,False,0,4
4,POINT (45 2.3987250992841447),POINT (53.4375 9.636338620241146),LINESTRING (45.1637069675639 2.541664115293592...,False,0,5
...,...,...,...,...,...,...
4094086,POINT (-45 -7.212658108949127),POINT (-47.8125 -4.801554594177967),LINESTRING (-45.055400500283824 -7.16553202138...,False,3068,3070
4094087,POINT (-45 -7.212658108949127),POINT (-45 -2.3987250992841447),"LINESTRING (-45 -7.11827481121803, -45 -7.0238...",False,3068,3071
4094088,POINT (-42.1875 -4.801554594177967),POINT (-47.8125 -4.801554594177967),LINESTRING (-42.297792946608595 -4.80200065705...,False,3069,3070
4094089,POINT (-42.1875 -4.801554594177967),POINT (-45 -2.3987250992841447),LINESTRING (-42.24280404901021 -4.754534435995...,False,3069,3071


# Graph

In [8]:
shape_restricted = union_all(
    borders.loc[1:3, 'geometry']
)

start = lonlat_to_healpix(
    airports[airports.ICAO=='UAAA'].geometry.x,
    airports[airports.ICAO=='UAAA'].geometry.y,
    depth,
    ellipsoid='WGS84',
)[0]
end = lonlat_to_healpix(
    airports[airports.ICAO=='KJFK'].geometry.x,
    airports[airports.ICAO=='KJFK'].geometry.y,
    depth,
    ellipsoid='WGS84',
)[0]

In [21]:
df['distance'] = df[['point_a', 'point_b']].apply(
    great_circle_distance,
    axis=1,
    raw=True,
    iterable=True,
)

In [22]:
df.to_csv('point_pairs_full.csv', index=False)